## 差分形式

In [1]:
# constants
Cat0 = (590, 569, 602)          # GtC initial CO2 in atmosphere
Coc0 = (1.4e5, 1.5e5, 1.6e5)          # GtC initial CO2 in ocean reservoir
Cveg0 = (540, 550, 560)         # GtC initial CO2 in vegetation reservoir
Cso0 = (1480, 1500, 1520)       # GtC initial CO2 in soil reservoir
T0 = (288, 288.15, 288.3)       # K initial average atmospheric temperature
kp = (0.175, 0.184, 0.193)      # yr-1 photosynthesis rate constant
kMM = 1.478                     # photosynthesis normalising constant
kc = (26* 1e-6, 29* 1e-6, 32* 1e-6)         # half-saturation point for photosynthesi
KM = (108* 1e-6, 120* 1e-6, 132* 1e-6)      # photosynthesis compensation point
ka = 1.773 * 1e20              # mole volume of atmosphere 
kr = (0.0828, 0.092, 0.1012)    # yr-1 plant respiration constant
kA = 8.7039 * 1e9               # plant respiration normalising constant 
Ea = (54.63e3, 54.83e3, 55.03e3)      # J mol−1 plant respiration activation energy
ksr =  (0.0303, 0.0337, 0.0371)   # yr−1 soil respiration rate constant
kB = 157.072                    #soil respiration normalising constant  
kt = (0.0828, 0.092, 0.1012)    # yr−1 turnover rate constant  
c = (4.22* 1e23, 4.69* 1e23, 5.16* 1e23)    # JK−1 specific heat capacity of Earth’s surface
aE = 5.101 *1e14                #  m2 Earth’s surface area
sigma = 5.67 * 1e-8             # Wm−2 K−4 Stefan-Boltzman constant
L_ = 43655                      #  mol−1 latent heat per mole of water  
R = 8.314                       # J mol−1 K molar gas constant
H = 0.5915                      # relative humidity  
A = (0.203, 0.225, 0.248)       # yr−1 surface albedo
S = (1231, 1368, 1504)          # Wm−2 solar flux 
tauCH4= (0.0208, 0.0231, 0.0254)# methane opacity 
P0  = (1.26* 1e11, 1.4* 1e11, 1.54* 1e11)   # Pa water vapor saturation constant  
F0 = (2.25* 1e-2, 2.5* 1e-2, 2.75* 1e-2)    # yr−1  ocean flux rate constant  
chi = (0.2, 0.3, 0.4)           # characteristic CO2 solubility 
zeta = (40, 50, 60)             # evasion factor
kappa = (0.02, 0.05, 0.2)       # yr−1 social learning rate 
beta = (0.5, 1, 1.5)            # net cost of mitigation 
delta = (0.5, 1, 1.5)           # strength of social norms
fmax  = (4,5,6)                 # maximum of warming cost function f(T) 
omega = (1, 3, 5)               # K nonlinearity of warming cost function f(T)  
Tc = (2.4, 2.5, 2.6)            # K critical temperature of f(T)  
tp = 10                         # yr previous years used for temperature projection  
tf = (0, 25, 50)                # yr years ahead for temperature projection
s = (30, 50, 70)                # yr half-saturation time for epsilon(t) from 2014
epsilon_max = (4.2, 7, 9.8)     # GtC yr−1  maximum change in epsilon(t) from 2014
x0 = (0.01, 0.05, 0.1)          # initial proportion of mitigators 
sec_to_year = 365.25*24*3600

In [6]:

def carbon_conc(Cat):
    pCO2a = 8.3259e13 * (Cat + Cat0[1])/ka
    return pCO2a

def carbon_uptake(T, pCO2a):
    if pCO2a >= kc[1] and -15 <= T and T <= 25:
        P = kp[1] * Cveg0[1] * kMM *(pCO2a - kc[1])/(KM[1]+pCO2a-kc[1])* (15+T)**2 *(25-T) / (5625)
        return P
    else:
        return 0

def respiration(T, Cveg, Cso):
    Rveg = kr[1] * Cveg * kA * np.exp(-Ea[1]/(R*(T+T0[1])))
    Rso = ksr[1] * Cso * kB * np.exp(-308.56/(T+T0[1]-227.13))

    return Rveg, Rso

def turnover(Cveg):
    L = kt[1] * Cveg
    return L

def ocean_flux(Cat, Coc):
    Foc = F0[1]*chi[1]*(Cat-zeta[1]*Cat0[1]*Coc/Coc0[1])
    return Foc

def atmosphere_dyna(pCO2a,T):
    tauCO2 = 1.73 * (pCO2a)**0.263
    tauH2O = 0.0126*(H*P0[1]*np.exp(-L_/(R*(T+T0[1]))))**0.503
    
    Fd = (1-A[1])*S[1]/4 *(1+0.75*(tauCO2+tauH2O+tauCH4[1]))
    return Fd

def f(T):
    return fmax[1]/(1+np.exp(-omega[1]*(T-Tc[1])))

def epsilon(t):
    epsilon2014 = 9.855
    return epsilon2014 + (t)*epsilon_max[1]/(t+s[1])

& dx = \kappa[1] \cdot x \cdot (1-x) \cdot (-\beta[1] + f(T) + \delta[1] \cdot (2x-1)) \quad \text{#assume } T_f = T \\
& dCat = \epsilon(t) \cdot (1-x) - P + R_{\text{veg}} + R_{\text{so}} - F_{\text{oc}} \\
& dCoc = F_{\text{oc}} \\
& dCveg = P - R_{\text{veg}} - L \\
& dCso = L - R_{\text{so}} \\
& dT = (F_d - \sigma \cdot (T+T_0[1])^4) \cdot \frac{aE}{c[1]} \\
& dCoc = \max(0, dCoc)

In [7]:
# calculate the variables with the equations
def calculate_vars(T, Cat, Cveg, Cso, Coc, t):
    # 只定义其中的相关过程
    pCO2a = carbon_conc(Cat)
    P = carbon_uptake(T, pCO2a)
    Rveg, Rso = respiration(T, Cveg, Cso)
    L = turnover(Cveg)
    Foc = ocean_flux(Cat, Coc)
    Fd = atmosphere_dyna(pCO2a, T)
    
    return P, Rveg, Rso, L, Foc, Fd

# update equations
def update(x, T, Rveg, Rso, Foc, L, P, Fd, t):
    
    # 这个可能是以后差分自己的写法操作，所以对应到相关的网格上操作了 
    dx = kappa[1] * x * (1-x) * (-beta[1] + f(T) + delta[1] * (2*x-1)) #assume Tf=T
    dCat = epsilon(t) *(1-x) - P + Rveg + Rso - Foc
    dCoc = Foc
    dCveg = P - Rveg - L
    dCso = L - Rso
    dT = (Fd - sigma * (T+T0[1])**4) * aE/c[1]
    dCoc = max(0, dCoc)
    
    return x+dx, Cat+dCat, Coc+dCoc, Cveg+dCveg, Cso+dCso, T+dT

In [19]:
#initialisation of variables
import numpy as np




x = x0[1]
T = 0
Cat = Cat0[1]
Cveg = Cveg0[1]
Cso = Cso0[1]
Coc = Coc0[1]
rangee = np.linspace(0, 100, 200)

index = 0
for t in rangee:
    index = index + 1
    # 先基本计算再进行更新
    P, Rveg, Rso, L, Foc, Fd = calculate_vars(T, Cat, Cveg, Cso, Coc, t)
    
    x, Cat, Coc, Cveg, Cso, T = update(x, T, Rveg, Rso, Foc, L, P, Fd, t)
    # 绘制上面P, Rveg, Rso, L, Foc, Fd的变量
    print(P, Rveg, Rso, L, Foc, Fd)
    print("-----------------------")
    print(x, Cat, Coc, Cveg, Cso, T)
    print("结束", index)

print(x, T)



120.87373881112681 50.59992170146056 50.550079794786306 50.6 -209.1075 398.31035675587117
-----------------------
0.04549406424631347 767.74601268512 150000.0 569.6738171096663 1500.0499202052138 8.067695510349512e-09
结束 1
124.60054022880303 52.409910108350196 50.551762142154715 52.4099911740893 -207.6169049048616 400.3604734826752
-----------------------
0.04135518486057788 963.1971886090364 150000.0 589.4544560560298 1501.9081492371483 1.836516611667353e-08
结束 2
127.42850433148678 54.22972612094504 50.61438460261572 54.22980995715474 -206.15102108543223 402.16781875219846
-----------------------
0.0375601225799455 1156.3434876107954 150000.0 608.4234243094168 1505.5235745916873 3.0628365493746366e-08
结束 3
129.65724766179196 55.97486855688312 50.736224681137834 55.97495503646634 -204.70242384291902 403.7938385868861
-----------------------
0.0340859596901081 1347.7817848260725 150000.0 626.1308483778593 1510.7623049470158 4.466007805574564e-08
结束 4
131.4637548210181 57.60394911849613 

## ode 形式